# Flatiron School Phase 3 Project

Student name: **Angelo Turri**

Student pace: **self paced**

Project finish date: **?**

Instructor name: **Mark Barbour**

Blog post URL: **?**

# Stakeholder

A charity organization has a list of Tanzanian water pumps. These pumps can either be entirely functional, functional with some defects in need of repair, or totally non-functional. This organization wants to fix as many of these pumps as it can. However, they have limited funds. To make the most of these funds, they need to be as efficient as possible – this means dispatching only what is necessary to each waterpoint to get the job done. In this hypothetical scenario, non functional water pumps require significantly more resources to repair than functional-needs-repair water pumps.

It is our job to use our available data to make predictions about the waterpoints that this charity organization gave us. The organization wants us to remember that non-functional waterpoints cost more to fix than functional-needs-repair waterpoints. We need to distinguish between them to the best of our ability.

# Data Origin & Description

The data was taken from **https://www.drivendata.org/competitions/7/pump-it-up-data-mining-the-water-table/data/**

Descriptions of any of the original variables can be found **below in the dictionary**. These descriptions are also provided here **https://www.drivendata.org/competitions/7/pump-it-up-data-mining-the-water-table/page/25/**.

# Rationale & Limitations

Why are you using **method**?

What about the current problem makes **method** suitable?

Assumptions of our model


Kind of data used

In [217]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold, RandomizedSearchCV
from sklearn.linear_model import LinearRegression, LogisticRegression, Ridge, Lasso
from sklearn.preprocessing import MinMaxScaler, StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.metrics import accuracy_score, get_scorer_names, confusion_matrix, classification_report
from sklearn.metrics import roc_curve, auc
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.ensemble import HistGradientBoostingClassifier, ExtraTreesClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.naive_bayes import GaussianNB, CategoricalNB
from sklearn.feature_selection import SelectKBest, chi2, mutual_info_classif
from xgboost import XGBClassifier
import xgboost as xgb
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from statistics import mode as md
from matplotlib import pyplot as plt
from IPython.display import clear_output, display_html 
from collections import Counter
from pprint import pprint
from itertools import product
import scipy.stats as ss

# Suppresses needless warnings
import warnings
warnings.filterwarnings("ignore")

In [218]:
# For aesthetics

class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

In [219]:
# Importing data
# testing is for the online competition only; it's not directly relevant to this project

testing = pd.read_csv("tanzanian_water_wells/X_test.csv")
X = pd.read_csv("tanzanian_water_wells/X_train.csv")
y = pd.read_csv("tanzanian_water_wells/y_train.csv")['status_group'].map({'functional': 2, 'functional needs repair': 1, 'non functional': 0})

idx = pd.read_csv("tanzanian_water_wells/X_test.csv")['id']

In [220]:
# Random state for any function in the notebook that uses one.
# This is for reproducibility.

state = 42

# Feature Selection

There are quite a few features in this dataset (40). The descriptions of each of these features was taken from Kaggle and listed below. We will not be using all the features, for several reasons:

- Some do not correlate with the target variable (status_group), e.g., the row ID;
- Others have too many categories;
- Some variables differ massively in their values from one dataset to another;
- Others are near copies of different variables in the same dataset, making it pointless to use them.

In [221]:
desc = {'amount_tsh': 'Total static head (amount water available to waterpoint)',
        'date_recorded': 'The date the row was entered',
        'funder': 'Who funded the well',
        'gps_height': 'Altitude of the well',
        'installer': 'Organization that installed the well', 
        'id': 'unique identifier of waterpoint',
        'longitude': 'GPS coordinate',
        'latitude': 'GPS coordinate',
        'wpt_name': 'Name of the waterpoint if there is one',
        'subvillage': 'Geographic location',
        'region': 'Geographic location',
        'region_code': 'Geographic location (coded)',
        'district_code': 'Geographic location (coded)',
        'lga': 'Geographic location',
        'ward': 'Geographic location',
        'population': 'Population around the well',
        'public_meeting': 'True/False',
        'recorded_by': 'Group entering this row of data',
        'scheme_management': 'Who operates the waterpoint',
        'scheme_name': 'Who operates the waterpoint',
        'permit': 'If the waterpoint is permitted',
        'construction_year': 'Year the waterpoint was constructed',
        'extraction_type': 'The kind of extraction the waterpoint uses',
        'extraction_type_group': 'The kind of extraction the waterpoint uses',
        'extraction_type_class': 'The kind of extraction the waterpoint uses',
        'management': 'How the waterpoint is managed',
        'management_group': 'How the waterpoint is managed',
        'payment': 'What the water costs',
        'payment_type': 'What the water costs',
        'water_quality': 'The quality of the water',
        'quality_group': 'The quality of the water',
        'quantity': 'The quantity of water',
        'quantity_group': 'The quantity of water',
        'source': 'The source of the water',
        'source_type': 'The source of the water',
        'source_class': 'The source of the water',
        'waterpoint_type': 'The kind of waterpoint',
        'waterpoint_type_group': 'The kind of waterpoint'
    }

### Variable inconsistency across train and test sets

We were given two datasets for analysis – one training dataset, complete with feature and target variables, and a testing dataset, which only had the feature variables (the target values were hidden).

Our goal is to accurately predict the target values in the test dataset. To do this successfully, we must use features in our model that correlate well with the target.

Unfortunately, some of our categorical variables differ in their categories from one dataset to another. Therefore, any model that uses these variables will have less success in the test dataset than in the training dataset - because it will come across numerous unknown categories.

It seems like the following variables should be removed from our features:

- wpt_name
- subvillage
- installer
- funder
- scheme_name
- ward
- date_recorded

In [222]:
differences = []

columns = list(X.select_dtypes(exclude=['float64', 'int64']).columns)

for col in columns:
    
    # This finds all the differences between two sets
    difference = set(list(X[col])) ^ set(list(testing[col]))
    differences.append(len(difference))
    
differences_df = pd.DataFrame({'column': list(columns), 'differences': differences})
differences_df = differences_df.sort_values(by=['differences'], ascending=False)
differences_df.head(10)

,column,differences
3,wpt_name,43128
5,subvillage,15120
2,installer,1584
1,funder,1403
12,scheme_name,1251
8,ward,145
0,date_recorded,51
14,extraction_type,1
23,quantity,0
21,water_quality,0


In [223]:
# Dropping all problem features from both test and train datasets

X = X.drop(list(differences_df['column'])[:7], axis=1)
testing = testing.drop(list(differences_df['column'])[:7], axis=1)

### Dropping variables that do not correlate with the target column

Two variables, namely "id" and "recorded_by", do not correlate with the target column (status_group).

- **id** is a unique numerical identifier for waterpoints; each waterpoint has a different identifier. An identifier such as this one cannot meaningfully correlate with the target column.

- **recorded_by** has only one value in the entire dataset, and therefore cannot correlate with the target column.

In [224]:
X.recorded_by.value_counts()

recorded_by
GeoData Consultants Ltd    59400
Name: count, dtype: int64

In [225]:
X = X.drop(['id', 'recorded_by'], axis=1)
testing = testing.drop(['id', 'recorded_by'], axis=1)

### Checking for collinearity among the variables

We want to eliminate collinearity. These variables might be strongly correlated with each other, based on the names they were given:

- **region** and **region_code**
- **scheme_management** and **scheme_name**
- **extraction_type**, **extraction_type_group** and **extraction_type_class**
- **management** and **management_group**
- **payment** and **payment_type**
- **water_quality** and **quality_group**
- **quantity** and **quantity_group**
- **source**, **source_type** and **source_class**
- **waterpoint_type** and **waterpoint_type_group**

I will search amongst the correlations between all of our features for evidence of collinearity.

In [226]:
# A helper function that makes any strong correlations easy to spot

def cts(val):
    if (val < -0.5):
        color = 'red'
    elif (val > 0.5):
        color = 'red'
    else: 
        color = 'black'
    return 'color: %s' % color

In [227]:
# Enables us to see all the correlations in this dataframe, as there are quite a few

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.concat([X, y], axis=1).apply(lambda x : pd.factorize(x)[0]).corr(method='pearson', min_periods=1).style.applymap(cts)

,amount_tsh,gps_height,longitude,latitude,num_private,basin,region,region_code,district_code,lga,population,public_meeting,scheme_management,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,status_group
amount_tsh,1.000000,0.225565,0.022839,0.022769,0.018267,0.097373,-0.066614,-0.040975,-0.077150,0.024900,0.111484,-0.004879,0.088875,0.082611,0.123520,-0.102274,-0.119237,-0.136671,0.045760,-0.024967,0.108294,0.108294,-0.069713,-0.075785,-0.129677,-0.129677,-0.014484,-0.014478,0.085647,-0.167683,-0.167616,-0.125189
gps_height,0.225565,1.000000,0.091864,0.091743,0.045389,0.089855,-0.082175,-0.044155,-0.048708,-0.031432,0.168467,-0.008090,0.068106,0.038948,0.334910,-0.138417,-0.149121,-0.156496,0.070697,0.028939,0.083959,0.083959,-0.058767,-0.057158,-0.088681,-0.088681,-0.082731,-0.089103,0.055094,-0.154355,-0.136941,-0.048207
longitude,0.022839,0.091864,1.000000,0.999900,0.014313,0.052439,0.009045,0.014896,-0.006607,0.022956,0.053131,0.016653,-0.007110,0.016481,0.039090,-0.028721,-0.031070,-0.035881,0.004882,0.006123,0.002333,0.002333,-0.035684,-0.019167,-0.000906,-0.000906,-0.035108,-0.033626,0.018191,-0.038435,-0.028578,-0.018156
latitude,0.022769,0.091743,0.999900,1.000000,0.014368,0.052505,0.009152,0.014998,-0.006647,0.022981,0.053122,0.016667,-0.007065,0.016526,0.039162,-0.028675,-0.031019,-0.035827,0.004932,0.006115,0.002340,0.002340,-0.035440,-0.019127,-0.000801,-0.000801,-0.035087,-0.033603,0.018242,-0.038376,-0.028512,-0.018109
num_private,0.018267,0.045389,0.014313,0.014368,1.000000,-0.041436,-0.051569,-0.049439,0.012700,0.060201,-0.004085,-0.004212,-0.017806,0.015779,0.028544,-0.031078,-0.030529,-0.025124,-0.006527,0.011049,-0.012531,-0.012531,-0.012167,-0.011056,-0.025757,-0.025757,-0.004469,-0.003768,0.038189,-0.045921,-0.041747,-0.007243
basin,0.097373,0.089855,0.052439,0.052505,-0.041436,1.000000,0.205807,0.221101,0.027690,0.175621,0.073288,0.094617,0.007797,0.127439,0.049935,0.091422,0.077460,0.115020,0.009402,-0.003203,0.056890,0.056890,0.020866,0.029922,0.004846,0.004846,0.241000,0.244420,0.133758,0.004052,-0.007917,0.009175
region,-0.066614,-0.082175,0.009045,0.009152,-0.051569,0.205807,1.000000,0.908184,0.056565,0.216317,-0.002812,0.005397,-0.046946,-0.072442,0.009994,0.021453,0.026072,0.052091,-0.031216,-0.008569,0.045160,0.045160,0.073083,0.072055,0.094505,0.094505,0.097848,0.122671,0.041678,-0.024325,-0.021727,0.075149
region_code,-0.040975,-0.044155,0.014896,0.014998,-0.049439,0.221101,0.908184,1.000000,0.045777,0.271859,0.020131,0.020502,-0.026403,-0.052544,0.046341,0.052062,0.033480,0.075424,-0.033733,0.012281,0.022480,0.022480,0.079717,0.082462,0.101111,0.101111,0.125013,0.151205,0.056925,-0.037729,-0.045512,0.066039
district_code,-0.077150,-0.048708,-0.006607,-0.006647,0.012700,0.027690,0.056565,0.045777,1.000000,0.168519,0.012793,0.016914,-0.105548,0.075358,0.007941,0.042588,0.039195,0.015551,-0.042632,-0.012413,-0.062036,-0.062036,0.024923,0.038838,0.042836,0.042836,-0.003339,-0.000609,-0.018465,0.045426,0.037503,0.046310
lga,0.024900,-0.031432,0.022956,0.022981,0.060201,0.175621,0.216317,0.271859,0.168519,1.000000,0.027292,0.037677,-0.152139,0.053310,0.008924,0.134864,0.147559,0.155971,-0.103805,-0.012734,-0.112748,-0.112748,0.027996,0.036732,0.019894,0.019894,0.115655,0.110658,-0.042610,0.078049,0.046462,0.048828


In [228]:
# Resets the custom visual settings we had in place for Pandas
# If we didn't reset them, pandas would attempt to show every row for large datasets
# This would crash our notebook

pd.reset_option('max_columns')
pd.reset_option('max_rows')

In [229]:
# Preprocessing for our training dataset

# Eliminating null values
X.scheme_management.fillna("None", inplace=True)
X.permit.fillna('Unknown', inplace=True)
X.public_meeting.fillna('Unknown', inplace=True)

# Casting certain features as different dtypes
X['permit'] = X['permit'].map({True: 'Yes', False: 'No', 'Unknown': 'Unknown'})
X['gps_height'] = X['gps_height'].astype('float64')
X['population'] = X['population'].astype('float64')
X['construction_year'] = X['construction_year'].astype('int64')
X['region_code'] = X['region_code'].astype('str')
X['district_code'] = X['district_code'].astype('str')

# !!IMPORTANT!!
# Separating features into numeric and categorical dataframes for preprocessing.
X_cat = X.select_dtypes(exclude=['float64', 'int64'])
X_cat = X_cat.astype('str')
oe = OrdinalEncoder()
oe.fit(X_cat)
X_cat = pd.DataFrame(oe.transform(X_cat), 
                     index = X_cat.index, 
                     columns = X_cat.columns)

## !!IMPORTANT!!
# Standardizing numercal features using MinMaxScaler
X_numeric = X.select_dtypes(['float64', 'int64'])
mms = MinMaxScaler()
mms.fit(X_numeric)
X_numeric = pd.DataFrame(mms.transform(X_numeric), 
                         columns = X_numeric.columns, 
                         index = X_numeric.index)

### Strong correlations

My threshold for a "strong correlation" is 0.5 here. I realize that is low for some people. However, I have a lot of features – too many at the moment, hence my aggressive approach.

These are the strong correlations we observed among our features:

- gps_height and construction_year
- scheme_management, management and management_group
- extraction, extraction_type_group and extraction_type_class
- payment and payment_type
- quantity and quantity_group
- source and source_type
- waterpoint_type and waterpoint_type_group

Of each of these groups, we must pick only one variable to keep.

### Correlation to status_group using chi-squared test of independence
How do we decide between these variables? I will use the chi-squared test for indepenence to determine how strongly the variables correlate with our target variable, "status_group." Whichever of these variables more strongly correlates with status_group will be used in our models.

In [230]:
groups = [['gps_height', 'construction_year'], 
          ['scheme_management', 'management', 'management_group'], 
          ['extraction_type', 'extraction_type_group', 'extraction_type_class'], 
          ['payment', 'payment_type'], 
          ['quantity', 'quantity_group'], 
          ['source', 'source_type'], 
          ['waterpoint_type', 'waterpoint_type_group']]

for group in groups:
    function = SelectKBest(score_func=chi2, k='all').fit(pd.concat([X_numeric, X_cat], axis=1)[group], y)
    function_results = pd.DataFrame({'feature': function.feature_names_in_, 'score': function.scores_}).sort_values(by=['score'], ascending=False).reset_index(drop=True)
    df = function_results.sort_values(by=['score'], ascending=False)
    display(df)

,feature,score
0,gps_height,172.710476
1,construction_year,66.593012


,feature,score
0,scheme_management,310.768525
1,management,176.680250
2,management_group,30.233213


,feature,score
0,extraction_type_class,4962.445269
1,extraction_type_group,3427.761791
2,extraction_type,2638.196579


,feature,score
0,payment,866.203572
1,payment_type,462.559171


,feature,score
0,quantity,672.865229
1,quantity_group,672.865229


,feature,score
0,source_type,568.229788
1,source,458.830986


,feature,score
0,waterpoint_type,3348.517448
1,waterpoint_type_group,2540.881101


This determines which variables we will keep, and which we will eliminate. The ones we will eliminate are:

- construction_year
- management
- management_group
- extraction_type_group
- extraction_type
- payment_type
- quantity_group (this choice doesn't matter, both are perfectly correlated with each other)
- source
- waterpoint_type_group

In [231]:
# Eliminating problematic categorical features
X_cat = X_cat.drop(['management', 'management_group', 
        'extraction_type_group', 'extraction_type', 'payment_type', 
        'quantity_group', 'source', 'waterpoint_type_group'], axis=1)

for col in X_cat.columns:
    X_cat[col]=X[col]

# Casting all categorical features as string dtypes for simplicity
X_cat = X_cat.astype('str')

# Combining both numeric and categorical features
X_cat_original = X_cat

X_cat = pd.get_dummies(X_cat, drop_first=True, dtype='int64')
# X_cat = X_cat.drop(['region_code_26.0'], axis=1)

# Eliminating a single problematic numerical feature
X_numeric = X_numeric.drop(['construction_year'], axis=1)

# Combining both numeric and categorical features
X = pd.concat([X_numeric, X_cat], axis=1)

In [287]:
combs = list(product(list(X_cat_original.columns) + ["status_group"], repeat=2))

combs_ps = {x: None for x in combs}
combs_vs = {x: None for x in combs}

for comb in combs:
    col1 = comb[0]
    col2 = comb[1]
    
    df = pd.concat([X_cat_original, y], axis=1)
    
    crosstab, test_results, expected = rp.crosstab(df[col1], df[col2],
                                               test= "chi-square",
                                               expected_freqs= True,
                                               prop= "cell")

    combs_ps[comb] = test_results['results'][1]
    combs_vs[comb] = test_results['results'][2]
    
df = pd.concat([X_cat_original, y], axis=1)
comb_df = {col: [None for i in df.columns] for col in df.columns}
comb_df = pd.DataFrame(comb_df, index=df.columns)

for v in list(combs_vs.keys()):
    comb_df[v[0]][v[1]] = combs_vs[v]
    
comb_df.style.applymap(cts)

,basin,region,region_code,district_code,lga,public_meeting,scheme_management,permit,extraction_type_class,payment,water_quality,quality_group,quantity,source_type,source_class,waterpoint_type,status_group
basin,1.000000,0.767200,0.774000,0.314000,0.907800,0.162700,0.246200,0.270500,0.250500,0.245000,0.119900,0.139200,0.139000,0.255000,0.123700,0.208600,0.127200
region,0.767200,1.000000,0.998100,0.325600,1.000000,0.358400,0.365500,0.446900,0.358400,0.357500,0.199000,0.215400,0.213100,0.356600,0.221200,0.294500,0.200900
region_code,0.774000,0.998100,1.000000,0.419600,0.972700,0.364800,0.381400,0.457100,0.369800,0.368800,0.208500,0.227500,0.232500,0.372700,0.257500,0.301400,0.208400
district_code,0.314000,0.325600,0.419600,1.000000,0.857600,0.187900,0.194200,0.242800,0.148800,0.167400,0.114600,0.130400,0.152600,0.165400,0.198300,0.120700,0.118700
lga,0.907800,1.000000,0.972700,0.857600,1.000000,0.580400,0.626600,0.865800,0.485100,0.563700,0.356300,0.395300,0.421500,0.508000,0.479700,0.423100,0.311500
public_meeting,0.162700,0.358400,0.364800,0.187900,0.580400,1.000000,0.252900,0.102600,0.106300,0.251800,0.115500,0.104600,0.146000,0.100300,0.056100,0.076400,0.056900
scheme_management,0.246200,0.365500,0.381400,0.194200,0.626600,0.252900,1.000000,0.227400,0.223700,0.211500,0.129200,0.140500,0.166800,0.277300,0.222000,0.191800,0.129400
permit,0.270500,0.446900,0.457100,0.242800,0.865800,0.102600,0.227400,1.000000,0.139800,0.159300,0.151600,0.145200,0.076200,0.162000,0.079100,0.136800,0.029600
extraction_type_class,0.250500,0.358400,0.369800,0.148800,0.485100,0.106300,0.223700,0.139800,1.000000,0.231400,0.147100,0.160100,0.108100,0.439600,0.271100,0.504400,0.241500
payment,0.245000,0.357500,0.368800,0.167400,0.563700,0.251800,0.211500,0.159300,0.231400,1.000000,0.133700,0.143300,0.127500,0.190300,0.098900,0.163700,0.182700
